In [3]:

import torch
import torch.backends.cudnn as cudnn
cudnn.deterministic = True


import os
import os.path
import pickle

import sys
sys.path.insert(0, "D:/OneDrive/연구/ML/MyProject/AgentNet") #PC
#importlib.reload(ML)
import src.DataStructure as DS
from src.utils import *
from src.system import *
from src.model import *

%matplotlib inline
#%matplotlib notebook
torch.set_printoptions(precision = 10)

In [ ]:
class DataGen():
    def __init__(self, system):
        pass
    
    def run(self, file_name, total_size, batch_size, train_ratio):

        train_image = []
        train_label = []
        test_image = []
        test_label = []
        
        batch_num = int(total_size/batch_size)
        train_size = int(batch_num * train_ratio)
        test_size = int(batch_num*(1-train_ratio))
        
        for i in range(train_size):
            data, answer = next(system)
            train_image.append(data.astype(float))
            train_label.append(answer)

        train_output = {'Image':train_image, 'Label':train_label}
        
        for i in range(test_size):
            data, answer = next(system)
            test_image.append(data.astype(float))
            test_label.append(answer)
        
        test_output = {'Image':test_image, 'Label':test_label}
            
        # Output pickle
        with open(file_name + '_train.pkl', 'wb') as f:
            pickle.dump(train_output, f)
            
        with open(file_name + '_test.pkl', 'wb') as f:
            pickle.dump(test_output, f)

In [ ]:
# Size parameters

single_data_size = 5000
total_data_size = single_data_size * 1
batch_size = 1
train_ratio = 0.8
filename = ''

batch_num = int(total_data_size/batch_size)
train_size = int(batch_num * train_ratio)
test_size = int(batch_num*(1-train_ratio))

In [ ]:
# Generating CA model dataset

pp = {}
system = CA()

pp['side_length'] = 14
pp['rule']={'alive': [2, 3], 'dead': [3]}
system.assign_pp(pp)

In [5]:
# Generating Vicsek model dataset

pp= {}
system = Vicsek_markovian()

pp['agent_num'] = 300
pp['neighbor_dist'] = 1
pp['neighbor_angle'] = 120
pp['noise_type'] = 'pos'
pp['noise_strength'] = 0.2
pp['grp_num'] = 1
system.assign_pp(pp)
file_name = system.rule_name
system.state_num = 4
system.answer_num = 1

In [ ]:
# Generating AOUP model dataset

pp = {}
system = AOUP()

pp['agent_num'] = 100
pp['dt'] = 1/100.
pp['data_step'] = 8
pp['label_step'] = 12
pp['state_num'] = 4
pp['answer_num'] = 1
pp['const_num'] = 1
system.assign_pp(pp)
system.assign_const(train_size, test_size, R_const_list = [[2.0, 4.0, 0.2], [2.0, 4.0, 0.2]])

In [ ]:
# Generating new dataset
file_name = system.rule_name 
generator = DataGen(system)
if not os.path.isfile(file_name+'_train.pkl'):
    generator.run(file_name, total_data_size, batch_size, train_ratio)
#generator.run(file_name, total_data_size, batch_size, train_ratio)